**РАБОТА С КОНЛЛУ-СИНТАГРУСОМ**

In [1]:
 ! pip install conllu
 from conllu import parse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
with open ('/content/ru_syntagrus-ud-dev1.conllu', 'r', encoding='utf-8') as f:
  data = f.read()
sentences = parse(data)

In [ ]:
len(sentences)

8906

In [3]:
sentence = sentences[0]
print(sentence)

TokenList<Алгоритм, ,, от, имени, учёного, аль, -, Хорезми, ,, -, точный, набор, инструкций, ,, описывающих, порядок, действий, исполнителя, для, достижения, результата, решения, задачи, за, конечное, время, ., metadata={sent_id: "2013Algoritm.xml_1", text: "Алгоритм, от имени учёного аль-Хорезми, - точный набор инструкций, описывающих порядок действий исполнителя для достижения результата решения задачи за конечное время."}>


In [4]:
token = sentence[12]
token

{'deprel': 'nmod',
 'deps': [('nmod:gen', 12)],
 'feats': {'Animacy': 'Inan',
  'Case': 'Gen',
  'Gender': 'Fem',
  'Number': 'Plur'},
 'form': 'инструкций',
 'head': 12,
 'id': 13,
 'lemma': 'инструкция',
 'misc': {'SpaceAfter': 'No'},
 'upos': 'NOUN',
 'xpos': None}

In [ ]:
sentence.serialize()
word = sentence.filter(upos='PROPN')
word

**МАСКИРОВАНИЕ**

In [ ]:
all_propn_sents = []
for sentence in sentences:
  sentence.serialize()
  word = sentence.filter(upos='PROPN')
  if word:
    met = sentence.metadata['text']
    all_propn_sents.append(met)

In [ ]:
len(all_propn_sents) #все предложения с propn списком

2815

In [ ]:
with open ('/content/sents_with_propn.txt', 'a', encoding='utf-8') as f:
  for sent in all_propn_sents:
    f.write(sent)
    f.write('\n')

In [ ]:
sentence.metadata

{'genre': 'fiction',
 'sent_id': '2020_RFFIInstrument_yazyka_1.xml_619',
 'text': 'В наши дни, когда никто никого не слушает, внимание - большая редкость.'}

In [ ]:
list_s = []
for i in range(0, len(sentences)):
  s = sentences[i]
  a = s.metadata['text']
  list_s.append(a)
list_s[:100] #все предложения списком

In [ ]:
propns = []
words = []
for sent in sentences:
  for token in sent:
    if token['upos'] == 'PROPN':
      propns.append(token['form'])     
print(propns) #все именованные сущности

for sent in sentences:
  for token in sent:
    words.append(token['form']) 
print(words) #все предложения списком по словам

In [ ]:
for i in propns:
  for s in range(len(words)):
    if i == words[s]:
      words[s] = '[MASK]'
print(words[:100])

['Алгоритм', ',', 'от', 'имени', 'учёного', 'аль', '-', '[MASK]', ',', '-', 'точный', 'набор', 'инструкций', ',', 'описывающих', 'порядок', 'действий', 'исполнителя', 'для', 'достижения', 'результата', 'решения', 'задачи', 'за', 'конечное', 'время', '.', 'В', 'старой', 'трактовке', 'вместо', 'слова', '"', 'порядок', '"', 'использовалось', 'слово', '"', 'последовательность', '"', ',', 'но', 'по', 'мере', 'развития', 'параллельности', 'в', 'работе', 'компьютеров', 'слово', '"', 'последовательность', '"', 'стали', 'заменять', 'более', 'общим', 'словом', '"', 'порядок', '"', '.', 'Это', 'связано', 'с', 'тем', ',', 'что', 'работа', 'каких-то', 'инструкций', 'алгоритма', 'может', 'быть', 'зависима', 'от', 'других', 'инструкций', 'или', 'результатов', 'их', 'работы', '.', 'Таким', 'образом', ',', 'некоторые', 'инструкции', 'должны', 'выполняться', 'строго', 'после', 'завершения', 'работы', 'инструкций', ',', 'от', 'которых', 'они', 'зависят']


**СОЗДАНИЕ ТЕКСТА ИЗ МАСКИРОВАННЫХ ПРЕДЛОЖЕНИЙ**

In [ ]:
words_text = ' '.join(words)
#print(words_text)

import re

words_text = words_text.replace(' .', '.')
words_text = words_text.replace('. " ', '. "')
words_text = words_text.replace(' ".', '".')

In [ ]:
a = re.sub('\s(?=[,.!?;:])', '', words_text)

regex = re.compile('\"\s\w+\s\"')
a = regex.sub(lambda match: match.group().replace(' ',""), a)

a = re.sub('\s(?=\))', '', a)
a = re.sub('(?<=\()\s', '', a)
#print(type(a))

a = a.replace('(" ', '("')
a = a.replace(' ")', ')"')
a = a.replace(' ",', '",')
a = a.replace('- " ', '- "')
a = a.replace('аль - ', 'аль-')
a = a.replace(' " ', ' "')
a[:3000]

In [ ]:
with open ('/content/masked_text.txt', 'a', encoding='utf-8') as f:
  for word in a:
    f.write(word) # файл со всеми предложениями где правильные маски

**РАБОТА С БЕРТОМ**

In [ ]:
!wget -c http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz
!tar -zxf rubert_cased_L-12_H-768_A-12_pt.tar.gz
!cp rubert_cased_L-12_H-768_A-12_pt/bert_config.json rubert_cased_L-12_H-768_A-12_pt/config.json
!pip install transformers
import transformers
tokenizer = transformers.BertTokenizerFast.from_pretrained('rubert_cased_L-12_H-768_A-12_pt')

config = transformers.BertConfig.from_pretrained('rubert_cased_L-12_H-768_A-12_pt', output_hidden_states=True)
model = transformers.BertForMaskedLM.from_pretrained('rubert_cased_L-12_H-768_A-12_pt', config=config)

In [ ]:
from tqdm.auto import tqdm
import torch

def get_bert_substs(text):
    text_encoded = tokenizer.encode(text, return_tensors="pt")
    texts_subtokens = tokenizer.convert_ids_to_tokens(text_encoded[0])

    mask_token_indices = torch.where(text_encoded == tokenizer.mask_token_id)
    model_pred = model(text_encoded, return_dict=True)

    token2model_ebedding_list = model_pred.hidden_states[-1]  # previous (-2 etc) elems are for intermediate layers
    
    mask_embedding = token2model_ebedding_list[mask_token_indices]
    mask_filler_candidates = tokenizer.convert_ids_to_tokens(model_pred.logits[mask_token_indices]
                                                                       .topk(100)
                                                                       .indices.flatten().tolist())
    return mask_filler_candidates, mask_embedding, texts_subtokens

**РАБОТА С ЗАМАСКИРОВАННЫМИ ПРЕДЛОЖЕНИЯМИ**

In [ ]:
text = []
with open ('/content/masked_text.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
text = fi.split('. ')
text

In [ ]:
text = ['{}.'.format(el) for el in text]
type(text)

list

In [ ]:
# BERT подставляет только на место первой маски
# -> выбираем только предложения с одной маской

In [ ]:
ms = []
withmask = []
import re
for s in text:
  if not re.search(r'.*\[MASK\].*\[MASK\].*', s):
    ms.append(s)
for s in ms:
  if re.search(r'.*\[MASK\].*', s):
    withmask.append(s)
withmask[:20] #список предложений только с одной маской

In [ ]:
#all_propn_sents - все предложения с propn списком
#withmask[:20] список предложений только с одной маской

lst_result = []
lst_nomask = []
for sent in all_propn_sents:
  beg = sent[:20]
  #print(beg)
  for el in withmask:
    if beg in el:
      lst_result.append(el)
      lst_nomask.append(sent)
    else:
      continue
lst_result[:20]

In [ ]:
lst_nomask[:25]

In [ ]:
with open ('/content/masks_check.txt', 'a', encoding='utf-8') as f:
  for sent in lst_result:
    f.write(sent) # файл где по 1 маске на предложение
    f.write('\n')

In [ ]:
with open ('/content/no_masks_check.txt', 'a', encoding='utf-8') as f:
  for sent in lst_nomask:
    f.write(sent) # файл где нет масок аналог с 1 маской на предложение
    f.write('\n')

In [ ]:
len(lst_nomask)

921

In [ ]:
len(lst_result) # полученный в итоге список предложений с которым я работаю

921

In [ ]:
withmask[:20]

In [ ]:
with open ('/content/file_again.txt', 'a', encoding='utf-8') as f:
  for sent in withmask:
    f.write(sent) # файл где по 1 маске на предложение
    f.write('\n')

In [ ]:
with open ('/content/masked_string.txt', 'a', encoding='utf-8') as f:
  for sent in masks:
    f.write(sent) # файл со всеми предложениями где правильные маски
    f.write('\n')

In [ ]:
len(masks) #было предложений всего с масками

2977

In [ ]:
len(withmask) #предложений где одна маска

1590

In [ ]:
d2 = {"id": 1948, "name": "Washer", "size": 3}
with open('out.txt','w') as out1:
    for key,val in d2.items():
        out1.write('{}: {}\n'.format(key,val))

**РАБОТА ПО АНАЛИЗУ МАСОК НА ПРИМЕРЕ ТЕКСТА ПРО ГАЛИЛЕО**

In [ ]:
import re

In [ ]:
galileo = []
with open ('/content/galileo.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
galileo = fi.split('\n')
len(galileo)

85

In [ ]:
galileo[84]

'nameВ настоящее время гипотеза [MASK] в среде историков считается бездоказательной и не обсуждается.'

In [ ]:
no_galileo = []
with open ('/content/no_galileo.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
no_galileo = fi.split('\n')
len(no_galileo)

85

In [ ]:
gal = []
with open ('/content/galileo_propns.txt', 'r', encoding='utf-8') as f:
  strs = f.read()

gal = strs.split('\n')
len(gal)

85

In [ ]:
# имеем категории: brand city cou geo name thing

In [ ]:
name_galileo = [] # все предложения с замаскированным NAME
for name in galileo:
    if "name" in name:
      name = re.sub(r'name', '', name)
      name_galileo.append(name)
    if "cou" in name:
      name = re.sub(r'cou', '', name)
      name_galileo.append(name)
    if "city" in name:
      name = re.sub(r'city', '', name)
      name_galileo.append(name)
    if "thing" in name:
      name = re.sub(r'thing', '', name)
      name_galileo.append(name)
    if "brand" in name:
      name = re.sub(r'brand', '', name)
      name_galileo.append(name)
    if "geo" in name:
      name = re.sub(r'geo', '', name)
      name_galileo.append(name)
    if name not in name_galileo:
      name_galileo.append(name)
name_galileo[83]

'- Не обнаружено свидетельств того, что найденный [MASK] донос имел какие-либо последствия.'

In [ ]:
reses_galileo = []
for sent in name_galileo:
  substs, mask_embedding, subtoks = get_bert_substs(sent)
  reses_galileo.append({
      "substs": substs,
      "sent": sent
      })
for i in reses_galileo:
  print(i, end='\n')

In [ ]:
len(reses_galileo)

85

In [ ]:
len(name_galileo)

85

In [ ]:
all = []
for i in range(len(name_galileo)):
  dct_galileo = {}
  dct_galileo["1_sentence"] = no_galileo[i]
  dct_galileo["2_masked_token"] = gal[i]
  dct_galileo["3_masked_sentence"] = name_galileo[i]
  dct_galileo["4_bert_hypotheses"] = reses_galileo[i]['substs'][:15]
  all.append(dct_galileo)
all

**ТЕКСТ ПРО АЛЬПИНИЗМ**

In [ ]:
alpin = []
with open ('/content/alpin.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
alpin = fi.split('\n')
len(alpin)

43

In [ ]:
no_alpin = []
with open ('/content/no_alpin.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
no_alpin = fi.split('\n')
len(no_alpin)

43

In [ ]:
alpin

In [ ]:
alp = []
with open ('/content/alpin_propns.txt', 'r', encoding='utf-8') as f:
  strs = f.read()

alp = strs.split('\n')
len(alp)

43

In [ ]:
reses_alpin = []
for sent in alpin:
  substs, mask_embedding, subtoks = get_bert_substs(sent)
  reses_alpin.append({
      "substs": substs,
      "sent": sent
      })
for i in reses_alpin:
  print(i, end='\n')

In [ ]:
for i in range(len(alp)):
  dct_alpin = {}
  dct_alpin["1_sentence"] = no_alpin[i]
  dct_alpin["2_masked_token"] = alp[i]
  dct_alpin["3_masked_sentence"] = alpin[i]
  dct_alpin["4_bert_hypotheses"] = reses_alpin[i]['substs'][:15]
  all.append(dct_alpin)
all

**ТЕКСТЫ ПРО ЭКОНОМИКУ**

In [ ]:
economics = []
with open ('/content/economics.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
economics = fi.split('\n')
len(economics)

57

In [ ]:
no_economics = []
with open ('/content/no_economics.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
no_economics = fi.split('\n')
len(no_economics)

57

In [ ]:
eco = []
with open ('/content/economics_propns.txt', 'r', encoding='utf-8') as f:
  strs = f.read()

eco = strs.split('\n')
len(eco)

57

In [ ]:
reses_eco = []
for sent in economics:
  substs, mask_embedding, subtoks = get_bert_substs(sent)
  reses_eco.append({
      "substs": substs,
      "sent": sent
      })
for i in reses_eco:
  print(i, end='\n')

In [ ]:
for i in range(len(eco)):
  dct_eco = {}
  dct_eco["1_sentence"] = no_economics[i]
  dct_eco["2_masked_token"] = eco[i]
  dct_eco["3_masked_sentence"] = economics[i]
  dct_eco["4_bert_hypotheses"] = reses_eco[i]['substs'][:15]
  all.append(dct_eco)
all

**ТЕКСТЫ ПРО БРЕНДЫ**

In [ ]:
brands = []
with open ('/content/brands.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
brands = fi.split('\n')
len(brands)

40

In [ ]:
no_brands = []
with open ('/content/no_brands.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
no_brands = fi.split('\n')
len(no_brands)

40

In [ ]:
bra = []
with open ('/content/brands_propns.txt', 'r', encoding='utf-8') as f:
  strs = f.read()

bra = strs.split('\n')
len(bra)

40

In [ ]:
reses_bra = []
for sent in brands:
  substs, mask_embedding, subtoks = get_bert_substs(sent)
  reses_bra.append({
      "substs": substs,
      "sent": sent
      })
for i in reses_bra:
  print(i, end='\n')

In [ ]:
for i in range(len(bra)):
  dct_brands = {}
  dct_brands["1_sentence"] = no_brands[i]
  dct_brands["2_masked_token"] = bra[i]
  dct_brands["3_masked_sentence"] = brands[i]
  dct_brands["4_bert_hypotheses"] = reses_bra[i]['substs'][:15]
  all.append(dct_brands)
all

**ТЕКСТЫ ПРО ИСТОРИЮ**

In [ ]:
history = []
with open ('/content/history.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
history = fi.split('\n')
len(history)

51

In [ ]:
no_history = []
with open ('/content/no_history.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
no_history = fi.split('\n')
len(no_history)

51

In [ ]:
his = []
with open ('/content/history_propns.txt', 'r', encoding='utf-8') as f:
  strs = f.read()

his = strs.split('\n')
len(his)

51

In [ ]:
reses_his = []
for sent in history:
  substs, mask_embedding, subtoks = get_bert_substs(sent)
  reses_his.append({
      "substs": substs,
      "sent": sent
      })
for i in reses_his:
  print(i, end='\n')

In [ ]:
for i in range(len(his)):
  dct_history = {}
  dct_history["1_sentence"] = no_history[i]
  dct_history["2_masked_token"] = his[i]
  dct_history["3_masked_sentence"] = history[i]
  dct_history["4_bert_hypotheses"] = reses_his[i]['substs'][:15]
  all.append(dct_history)
all

**ТЕКСТЫ ПРО ПРИРОДУ**

In [ ]:
nature = []
with open ('/content/nature.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
nature = fi.split('\n')
len(nature)

66

In [ ]:
no_nature = []
with open ('/content/no_nature.txt', 'r', encoding='utf-8') as f:
  fi = f.read()
no_nature = fi.split('\n')
len(no_nature)

66

In [ ]:
nat = []
with open ('/content/nature_propns.txt', 'r', encoding='utf-8') as f:
  strs = f.read()

nat = strs.split('\n')
len(nat)

66

In [ ]:
no_nature

In [ ]:
reses_nature = []
for sent in nature:
  substs, mask_embedding, subtoks = get_bert_substs(sent)
  reses_nature.append({
      "substs": substs,
      "sent": sent
      })
for i in reses_nature:
  print(i, end='\n')

In [ ]:
for i in range(len(nat)):
  dct_nature = {}
  dct_nature["1_sentence"] = no_nature[i]
  dct_nature["2_masked_token"] = nat[i]
  dct_nature["3_masked_sentence"] = nature[i]
  dct_nature["4_bert_hypotheses"] = reses_nature[i]['substs'][:15]
  all.append(dct_nature)
all

**ЗАПИСЬ СПИСКА СЛОВАРЕЙ В ФАЙЛ**

In [ ]:
with open('list словарей.txt','a') as f:
  for el in all:
    for key,val in el.items():
      f.write('{}:{}\n'.format(key,val))
    f.write('\n')

In [ ]:
len(all)

342

In [ ]:
import csv
path = '/content/list2.csv'
def save(all, path):#ФУНКЦИЯ ЗАПИСИ СПАРСЕННЫХ ДАННЫХ В CSV ФАЙЛ
    with open(path,'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('ne','yu','jk', 'ol'))

        for team in all:
            writer.writerow((team['ne'],team['yu'],team['jk'], team['ol']))

In [ ]:
with open('dct_5.csv', 'a', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    for el in all:
      for key, value in el.items():
        writer.writerow([key, value])
      writer.writerow('\n')